# **Paso 0: Gestión de dependencias**
### **0.0. Instalar librerias**

In [ ]:
!pip install --upgrade openai

### **0.1. Importar librerias**

In [ ]:
import os
import openai
import re
import json
import pandas as pd
import string
import requests
import random

### **0.2. Configurar openAI**

In [ ]:
# openai.organization = ""
openai.api_key = "<api-key>"
%env OPENAI_API_KEY=<api-key>

# **Paso 1: Preparar Data**

## **1.1. Dataset**
En formato .json .csv

## **1.2. Convertir a .jsonl**

In [ ]:
# !openai tools fine_tunes.prepare_data -f respuestas-sarcasticas.json
!openai tools fine_tunes.prepare_data -f haikus.csv

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 42 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `\nResp:`
- All prompts start with prefix `1: `
- All completions end with suffix ` END`

Based on the analysis we will perform the following actions:
- [Necessary] Your format `CSV` will be converted to `JSONL`


Your data will be written to a new JSONL file. Proceed [Y/n]: y

Wrote modified file to `haikus_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "haikus_prepared.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `\nResp:` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=[" E

# **Paso 3: Crear modelo fine-tuned**

In [ ]:
!openai api fine_tunes.create -t "haikus_prepared.jsonl" -m "davinci"
# !openai api fine_tunes.follow -i <token> #solo se usa cuando se corta el proceso y openai, nos devuelve el <token> para reanudar el proceso de fine-tuning

Upload progress: 100% 5.49k/5.49k [00:00<00:00, 9.94Mit/s]
Uploaded file from haikus_prepared.jsonl: file-Mr4pf7nLRNyGNvWGrHQKhvxj
Created fine-tune: ft-IMvvDTCdH6sMFlXGGDdxIDyy
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-09-02 07:56:57] Created fine-tune: ft-IMvvDTCdH6sMFlXGGDdxIDyy
[2023-09-02 07:57:06] Fine-tune costs $0.22
[2023-09-02 07:57:06] Fine-tune enqueued. Queue number: 0
[2023-09-02 07:57:07] Fine-tune started



In [ ]:
openai.FineTune.list() #saber que modelos tengo finetuneado

<OpenAIObject list at 0x7a28589fb0b0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine-tune",
      "id": "ft-IMvvDTCdH6sMFlXGGDdxIDyy",
      "hyperparams": {
        "n_epochs": 4,
        "batch_size": 1,
        "prompt_loss_weight": 0.01,
        "learning_rate_multiplier": 0.1
      },
      "organization_id": "org-Fi06OLn1iMDgxfNR42XdZXMK",
      "model": "davinci",
      "training_files": [
        {
          "object": "file",
          "id": "file-Mr4pf7nLRNyGNvWGrHQKhvxj",
          "purpose": "fine-tune",
          "filename": "haikus_prepared.jsonl",
          "bytes": 5489,
          "created_at": 1693641417,
          "status": "processed",
          "status_details": null
        }
      ],
      "validation_files": [],
      "result_files": [
        {
          "object": "file",
          "id": "file-zEch50t6DvVIbuPue4VG2hBI",
          "purpose": "fine-tune-results",
          "filename": "compiled_results.csv",
          "bytes": 8445,
          "

# **Paso 4: Usar el modelo fine-tuned**

In [ ]:
def completion(prompt, engine_id, debug=True):

  COMPLETION_ENDPOINT = 'https://api.openai.com/v1/completions'

  headers = {'Authorization': 'Bearer {api_key}'.format(api_key=openai.api_key),
              'Content-Type': 'application/json'}

  data = {
      # "messages": [{"role": "user", "content": prompt}],
      "prompt": prompt,
      "model": engine_id,
      "stop": [" END"]
  }

  response = requests.post(COMPLETION_ENDPOINT, headers=headers, data=json.dumps(data))
  result = response.json()

  if debug:
    print("Headers:")
    print(json.dumps(headers, ident=4))
    print("Data:")
    print(json.dumps(data, ident=4))
    print("Result:")
    print(json.dumps(result, ident=4))

  if response.status_code == 200:
    return[x['text'].strip() for x in result['choices']]
  else:
    return "Error: {}".format(result['error']['message'])

In [ ]:
MY_MODEL = "davinci:ft-personal-2023-09-02-08-03-21"
MY_PROMPT = "1: agua\ncielo\nmontaña\nResp:"

In [ ]:
completion(MY_PROMPT, MY_MODEL, debug=False)

['El agua en la quebrada\nredonda como cielo']